# [Variables](https://www.tensorflow.org/programmers_guide/variables)

A ```tf.Variable``` 
- represents a tensor whose value can be changed by running ops on it. 
- unlike ```tf.Tensor``` objects, a ```tf.Variable``` exists outside the context of a single session.run call. Multiple workers can see the same values for a ```tf.Variable```.

## Creating a variable
The best way to create a variable is to call the ```tf.get_variable``` function. 
- requires you to specify the variable's name. 
- allows you to reuse a previously created variable of the same name, making it easy to define models which reuse layers.

By default, have the dtype ```tf.float32``` and its initial value will be randomized via ```tf.glorot_uniform_initializer```.
```python
my_variable = tf.get_variable("my_variable", [1, 2, 3]) # (name, shape)
```
Optionally specify the ```dtype``` and initializer to ```tf.get_variable```.
```python
my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32,
  initializer=tf.zeros_initializer)
```
To have the value of a ```tf.Tensor```.
```python
other_variable = tf.get_variable("other_variable", dtype=tf.int32, initializer=tf.constant([23, 42]))
```

### Variable collections
Tensorflow provides **collections** to have a single way to access all of tensors or other objects, such as ```tf.Variable``` instances.

By default every tf.Variable gets placed in the following two collections: 
- ```tf.GraphKeys.GLOBAL_VARIABLES```: variables that can be shared across multiple devices. 
- ```tf.GraphKeys.TRAINABLE_VARIABLES```: variables for which TensorFlow will calculate gradients.

If you don't want a variable to be trainable, 
- add it to the ```tf.GraphKeys.LOCAL_VARIABLES``` collection instead. For example,
```python
my_local = tf.get_variable("my_local", shape=(), collections=[tf.GraphKeys.LOCAL_VARIABLES])
```
- Alternatively, you can specify ```trainable=False``` as an argument to ```tf.get_variable```.
```python
my_non_trainable = tf.get_variable("my_non_trainable", shape=(), trainable=False)
```

You can also use your own collections. Any string is a valid collection name, and there is no need to explicitly create a collection.
```python
tf.add_to_collection("my_collection_name", my_local)
```
And to retrieve a list of all the variables (or other objects) you've placed in a collection you can use:
```python
tf.get_collection("my_collection_name")
```
### Device placement
come later.

## Initializing variables
Variables must be initialized. 
- Most high-level frameworks such as ```tf.contrib.slim```, ```tf.estimator.Estimator``` and Keras automatically initialize variables for you before training a model.
- Explicit initialization is otherwise useful. It
  - avoid potentially expensive initializers 
  - allow determinism when randomly-initialized variables are shared in a distributed setting
- ```tf.global_variables_initializer()``` initializes all trainable variables. This function initializes all variables in the ```tf.GraphKeys.GLOBAL_VARIABLES``` collection.
```python
session.run(tf.global_variables_initializer())
# Now all variables are initialized.
```
- Initialize variables explicitly
```python
session.run(my_variable.initializer)
```
- Check which variables have not been initialized
```python
print(session.run(tf.report_uninitialized_variables()))
```
- Use ```variable.initialized_value()```, any time the value of a variable in a context in which not all variables are initialized (say, if you use a variable's value while initializing another variable)
```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = tf.get_variable("w", initializer=v.initialized_value() + 1)
```

## Using variables
To use the value of a ```tf.Variable``` in a TensorFlow graph, simply treat it like a normal ```tf.Tensor```:
```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
w = v + 1  # w is a tf.Tensor which is computed based on the value of v.
           # Any time a variable is used in an expression it gets automatically
           # converted to a tf.Tensor representing its value.
```
To assign a value to a variable, use the methods ```assign```, ```assign_add```, and friends in the ```tf.Variable``` class.
```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
tf.global_variables_initializer().run()
sess.run(assignment)  # or assignment.op.run(), or assignment.eval()
```

To force a re-read of the value of a variable after something has happened, use ```tf.Variable.read_value```.
```python
v = tf.get_variable("v", shape=(), initializer=tf.zeros_initializer())
assignment = v.assign_add(1)
with tf.control_dependencies([assignment]):
  w = v.read_value()  # w is guaranteed to reflect v's value after the
                      # assign_add operation.
```

## Sharing variables
TensorFlow supports two ways of sharing variables.
- Explicitly passing ```tf.Variable``` objects around.
- Implicitly wrapping ```tf.Variable``` objects within ```tf.variable_scope``` objects.

Variable scopes allow you to control variable reuse when calling functions which implicitly create and use variables.

Suppose, we want to create a convolutional/relu layer
```python
def conv_relu(input, kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)
```
If we want many such convolutional layers, calling this function repeatedly wouldn't work.
```python
input1 = tf.random_normal([1,10,10,32])
input2 = tf.random_normal([1,20,20,32])
x = conv_relu(input1, kernel_shape=[5, 5, 32, 32], bias_shape=[32])
x = conv_relu(x, kernel_shape=[5, 5, 32, 32], bias_shape = [32])  # This fails.
```
creating new "weights" and "biases" variables or reuse the existing ones is unclear. 

Calling ```conv_relu``` in scopes solves this ambiguity.
- create new variables
```python
def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])
```

- want the variables to be shared
  - use ```reuse=True```
  ```python
  with tf.variable_scope("model"):
      output1 = my_image_filter(input1)
  with tf.variable_scope("model", reuse=True):
      output2 = my_image_filter(input2)
  # or
  with tf.variable_scope("model") as scope:
      output1 = my_image_filter(input1)
  with tf.variable_scope(scope, reuse=True): # initialize a variable scope based on another one
      output3 = my_image_filter(input2)
  ```
  - call ```scope.reuse.variables()``` to trigger a reuse
  ```python
  with tf.variable_scope("model") as scope:
      output1 = my_image_filter(input1)
      scope.reuse_variables()
      output2 = my_image_filter(input2)
  ```

In [104]:
import tensorflow as tf
import numpy as np

# tf.reset_default_graph()

# placeholder for batch pairs of observations and actions
tf_data = tf.placeholder(shape = None, name = "data", dtype = tf.float32)
tf_lens = tf.placeholder(shape = [None], name = "lens", dtype = tf.int32)

# tf_lens = tf.constant(5)

# tf_first = tf.split(tf_data, tf_lens)

#v = tf.get_variable("v", shape=(2,), initializer=tf.constant_initializer([2.0, 3.0]))
#w = tf.get_variable("w", shape=(), initializer=tf.constant_initializer(1.0))
# tf.slice(v, [0], [1]) = w

#y = tf.slice(v, [0], [1])[0]
#l1 = [y+1, w]
#l2 = [y+w, y*w]

# l3 = [l1, l2]

#z = tf.nn.softmax(l1)

#assignment = v.assign_add([2.0, 2.0])

a = np.arange(10)
lens = [2,2,2,2,2]

print(a)
print(lens)

sess = tf.Session()
sess.__enter__() # equivalent to `with sess:`
sess.run(tf.global_variables_initializer())
    # print([y.eval(), w.eval()])
    # print(z.eval())
feed_dict = {tf_lens:lens}
    # feed_dict = {tf_data:a}
y = sess.run([tf_lens], feed_dict = feed_dict)
    #first = sess.run(tf_first, feed_dict = feed_dict)
    #print(np.array(first))
    # print(tf_data_group.eval())
    
print(y[0])
    
tf_first = tf.split(tf_data, y[0])
#sums = []
#for i in range(len(tf_first)):
#    tf_sum = tf.reduce_sum(tf_first[i], 0)
#    sums.append(tf_sum)

#soft_sums = tf.nn.softmax(sums)
z = sess.run(tf.reduce_sum(tf_first, 1), feed_dict = {tf_data:a})
print(z)

x = tf.constant([[1, 1, 1], [1, 1, 1]])
g = tf.reduce_sum(x)
print(sess.run(g))
import math

[0 1 2 3 4 5 6 7 8 9]
[2, 2, 2, 2, 2]
[2 2 2 2 2]
[ 1.  5.  9. 13. 17.]
6
